In [27]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
import missingno as msno 
import seaborn as sns
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.model_selection import train_test_split


datos = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/data-preprocessing-project-tutorial/main/AB_NYC_2019.csv")
#datos.head()



#datos.info()
#datos.nunique()

#Con datos.info() ya veo que hay varias columnas con Nans: name, host_name, last_review y reviews_per_month.

datos_util = datos.drop(["name", "host_id", "latitude", "longitude", "host_name", "last_review", "reviews_per_month", "calculated_host_listings_count"], axis = 1)

minn_est = datos_util["minimum_nights"].describe()
print(minn_est)

nights_iq = minn_est["75%"] - minn_est["25%"]

slimit_minn = minn_est["75%"] + 1.5 * nights_iq
ilimit_minn = minn_est["25%"] - 1.5 * nights_iq

print(f"Los valores para noches mínimas son:{slimit_minn} y {ilimit_minn}")

price_est = datos_util["price"].describe()
print(price_est)

price_iq = price_est["75%"] - price_est["25%"]

slimit_price = price_est["75%"] + 1.5 * price_iq
ilimit_price = price_est["25%"] - 1.5 * price_iq

print(f"Los valores para precio son:{slimit_price} y {ilimit_price}")

number_of_reviews_est = datos_util["number_of_reviews"].describe()
print(number_of_reviews_est)

number_of_reviews_iq = number_of_reviews_est["75%"] - number_of_reviews_est["25%"]

slimit_number_of_reviews = number_of_reviews_est["75%"] + 1.5 * number_of_reviews_iq
ilimit_number_of_reviews = number_of_reviews_est["25%"] - 1.5 * number_of_reviews_iq

print(f"Los valores para reviews son:{slimit_number_of_reviews} y {ilimit_number_of_reviews}")

#Solo vamos a utilizar los límites superiores porque los inferiores son siempre negativos y no tienen sentido:
#Eliminamlos los outlayers:

datos_util = datos_util[datos_util["minimum_nights"] <= 11]
datos_util = datos_util[datos_util["price"] <= 334]
datos_util = datos_util[datos_util["number_of_reviews"] <= 58.5]

#Normalización de variables que afectan al precio:


datos_util["neighbourhood_group"] = pd.factorize(datos_util["neighbourhood_group"])[0]
datos_util["room_type"] = pd.factorize(datos_util["room_type"])[0]

num_variables = ["number_of_reviews", "minimum_nights", "availability_365", "neighbourhood_group", "room_type"]
scaler = MinMaxScaler()
scal_features = scaler.fit_transform(datos_util[num_variables])
df_scal = pd.DataFrame(scal_features, index = datos_util.index, columns = num_variables)
df_scal["price"] = datos_util["price"]
df_scal.head()


#Train y test

X = df_scal.drop("price", axis=1)
y = df_scal["price"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

selection_model = SelectKBest(score_func=f_regression)  
selection_model.fit(X_train, y_train)
ix = selection_model.get_support()

X_train_sel = pd.DataFrame(selection_model.transform(X_train), columns=X_train.columns.values[ix])
X_test_sel = pd.DataFrame(selection_model.transform(X_test), columns=X_test.columns.values[ix])

X_train_sel.head()


X_train_sel["price"] = list(y_train)
X_test_sel["price"] = list(y_test)
X_train_sel.to_csv("../data/processed/clean_train.csv", index = False)
X_test_sel.to_csv("../data/processed/clean_test.csv", index = False)


count   48895.00
mean        7.03
std        20.51
min         1.00
25%         1.00
50%         3.00
75%         5.00
max      1250.00
Name: minimum_nights, dtype: float64
Los valores para noches mínimas son:11.0 y -5.0
count   48895.00
mean      152.72
std       240.15
min         0.00
25%        69.00
50%       106.00
75%       175.00
max     10000.00
Name: price, dtype: float64
Los valores para precio son:334.0 y -90.0
count   48895.00
mean       23.27
std        44.55
min         0.00
25%         1.00
50%         5.00
75%        24.00
max       629.00
Name: number_of_reviews, dtype: float64
Los valores para reviews son:58.5 y -33.5
